# ML_spike
- Script pour créer un modèle de prédiction
- Auteur : Benjamin ALLEAU / benjamin.fraime@gmail.com

# Package

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

In [11]:
# Charger le fichier
df = pd.read_csv("data_preprocess/df_resultat_expanded.csv")

In [12]:
df.head(3)

,Num Court,Joueur,Coéquipier,Adversaires,Diff Score,Résultat,week
0,1,Théophane F.,Clement L.,"Simon P., Luis A.",-1,Perdu,1
1,1,Clement L.,Théophane F.,"Simon P., Luis A.",-1,Perdu,1
2,1,Simon P.,Luis A.,"Théophane F., Clement L.",1,Gagné,1


In [13]:
# Sélection des colonnes pertinentes
df = df[["Num Court", "Joueur", "Coéquipier", "Adversaires", "Résultat"]].copy()

# Convertir les adversaires en deux colonnes distinctes
df[["Adversaire_1", "Adversaire_2"]] = df["Adversaires"].str.split(", ", expand=True)
df.drop(columns=["Adversaires"], inplace=True)

# Convertir "Num Court" en entier
df["Num Court"] = pd.to_numeric(df["Num Court"], errors='coerce')
df.dropna(subset=["Num Court"], inplace=True)  # Supprimer les lignes avec des valeurs non valides
df["Num Court"] = df["Num Court"].astype(int)  # Assurer un type entier

# Calcul du terrain moyen pour chaque joueur
terrain_moyen = df.groupby("Joueur")["Num Court"].mean()

# Calcul du taux de victoire moyen pour chaque joueur
taux_victoire = df.groupby("Joueur")["Résultat"].apply(lambda x: (x == "Gagné").mean())

def get_terrain_moyen(joueur):
    return terrain_moyen.get(joueur, df["Num Court"].mean())  # Valeur moyenne si joueur inconnu

def get_taux_victoire(joueur):
    return taux_victoire.get(joueur, 0.5)  # 50% par défaut si inconnu

# Ajouter les terrains moyens et taux de victoire des équipes
df["Terrain_Moyen_Equipe"] = df[["Joueur", "Coéquipier"]].apply(lambda row: np.mean([get_terrain_moyen(row["Joueur"]), get_terrain_moyen(row["Coéquipier"])]), axis=1)
df["Terrain_Moyen_Adversaires"] = df[["Adversaire_1", "Adversaire_2"]].apply(lambda row: np.mean([get_terrain_moyen(row["Adversaire_1"]), get_terrain_moyen(row["Adversaire_2"])]), axis=1)
df["Taux_Victoire_Equipe"] = df[["Joueur", "Coéquipier"]].apply(lambda row: np.mean([get_taux_victoire(row["Joueur"]), get_taux_victoire(row["Coéquipier"])]), axis=1)
df["Taux_Victoire_Adversaires"] = df[["Adversaire_1", "Adversaire_2"]].apply(lambda row: np.mean([get_taux_victoire(row["Adversaire_1"]), get_taux_victoire(row["Adversaire_2"])]), axis=1)

# Sélection des variables d'entrée
X = df[["Terrain_Moyen_Equipe", "Terrain_Moyen_Adversaires", "Taux_Victoire_Equipe", "Taux_Victoire_Adversaires"]]
y = df["Résultat"].map({"Gagné": 1, "Perdu": 0})  # 1 = victoire, 0 = défaite

# Séparer les données en train et test
s80 = 440
X_train = X.iloc[:s80]
X_test = X.iloc[s80:]
y_train = y.iloc[:s80]
y_test = y.iloc[s80:]

# Optimisation des hyperparamètres avec GridSearchCV
param_grid = {
    'n_estimators': [800],
    'max_depth': [None, 2, 4,8],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [2, 4]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Meilleur modèle optimisé
model = grid_search.best_estimator_
print(model)

# Prédictions et évaluation
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Précision sur les données d'entraînement : {train_accuracy * 100:.2f}%")
print(f"Précision sur les données de test : {test_accuracy * 100:.2f}%")

# Sauvegarde du modèle
joblib.dump(model, "models/match_prediction_model.pkl")
joblib.dump((terrain_moyen, taux_victoire), "models/terrain_taux_victoire.pkl")


Fitting 5 folds for each of 16 candidates, totalling 80 fits
RandomForestClassifier(max_depth=4, min_samples_leaf=2, min_samples_split=10,
                       n_estimators=800, random_state=42)
Précision sur les données d'entraînement : 87.73%
Précision sur les données de test : 75.00%


['models/terrain_taux_victoire.pkl']

In [14]:
# Fonction pour prédire un match
def predire_match(joueur1, coequipier, adversaire1, adversaire2):
    """Prédit la probabilité de victoire en utilisant les terrains moyens et taux de victoire des équipes."""
    
    # Charger le modèle et les données
    model = joblib.load("models/match_prediction_model.pkl")
    terrain_moyen, taux_victoire = joblib.load("models/terrain_taux_victoire.pkl")
    
    # Récupérer les terrains moyens et taux de victoire
    terrain_equipe = np.mean([get_terrain_moyen(joueur1), get_terrain_moyen(coequipier)])
    terrain_adversaires = np.mean([get_terrain_moyen(adversaire1), get_terrain_moyen(adversaire2)])
    taux_victoire_equipe = np.mean([get_taux_victoire(joueur1), get_taux_victoire(coequipier)])
    taux_victoire_adversaires = np.mean([get_taux_victoire(adversaire1), get_taux_victoire(adversaire2)])
    
    # Créer l'entrée
    input_data = np.array([[terrain_equipe, terrain_adversaires, taux_victoire_equipe, taux_victoire_adversaires]])
    
    # Prédiction
    prob = model.predict_proba(input_data)[0][1]  # Probabilité de victoire
    
    return f"Probabilité de victoire de {joueur1} et {coequipier} contre {adversaire1} et {adversaire2} : {prob * 100:.2f}%"

# Exemple d'utilisation
print(predire_match("Théophane F.", "Clément L.", "Luis A.", "Nil M."))

Probabilité de victoire de Théophane F. et Clément L. contre Luis A. et Nil M. : 55.38%


c:\Users\benal\Documents\travail\spike\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [15]:
# Exemple d'utilisation
print(predire_match( 'Théophane F.', "Clément L.", "Caroline N.", "Benjamin A."))

c:\Users\benal\Documents\travail\spike\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Probabilité de victoire de Théophane F. et Clément L. contre Caroline N. et Benjamin A. : 78.46%


In [16]:
df.head(4)

,Num Court,Joueur,Coéquipier,Résultat,Adversaire_1,Adversaire_2,Terrain_Moyen_Equipe,Terrain_Moyen_Adversaires,Taux_Victoire_Equipe,Taux_Victoire_Adversaires
0,1,Théophane F.,Clement L.,Perdu,Simon P.,Luis A.,1.1750,1.0625,0.700000,0.541667
1,1,Clement L.,Théophane F.,Perdu,Simon P.,Luis A.,1.1750,1.0625,0.700000,0.541667
2,1,Simon P.,Luis A.,Gagné,Théophane F.,Clement L.,1.0625,1.1750,0.541667,0.700000
3,1,Luis A.,Simon P.,Gagné,Théophane F.,Clement L.,1.0625,1.1750,0.541667,0.700000
